#### DNN 기반 회귀 모델 구현
- 사용되는 데이터셋 : iris.csv
- feature : 3개
- target : 1개 
- 학습방법 : 지도학습 -> 회귀
- 알고리즘 : 인공신경망(ANN) -> 심층(은닉층) 신경망 -> MLP(층이여러개), DNN(은닉층이 많은 구성) 
- FramWork : Pytorch


[1] 모듈  로딩 & 데이터  준비 <HR>
학습스케쥴링  
    -학습 시 동적으로 lr값을 조절해주는 모듈
    

In [43]:
# 모듈 로딩
# 모델관련 모듈
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

import torch.optim as optim # 최적화 관련 모듈
import torch.optim.lr_scheduler as lr_scheduler # 최적화 스케쥴링 관련 모듈
from torchmetrics.regression import R2Score, MeanSquaredError
from torchinfo import summary

# 데이터 전처리 및 시각화 모듈
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split



In [44]:
# 활용 패키지 버전 체크
def versioncheck():
    print(f' torch  {torch.__version__}')
    print(f' pandas  {pd.__version__}')

versioncheck()

 torch  2.4.1
 pandas  2.0.3


In [45]:
DATA_FILE = r'C:\Users\zizonkjs\머신러닝,딥러닝\data\iris.csv'
irisdf=pd.read_csv(DATA_FILE, usecols=[0,1,2,3])
irisdf

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


[2] - 모델 클레스 설계 및 정의 - <hr>
어떤 클래스를 만들까? 고려
클래스 목적 : iris.DataSet 학습 후 추론
클래스 이름 : IrisRegModel
부모 클래스 : nn.Module
매개 변수   : 층별 입출력 갯수 고정하기 때문에 필요 없음
클래스 속성 : featureDF, targetDF, n_rows, n_features
클래스 기능 : __init__() : 모델 구조, forward() : 순방향 학습 <= 오버라이딩(overriding) 상속관계일 때

클래스 구조  
    -입력층 : 피쳐 3개  퍼셉트론 : 10개(보통 입력 때 많이 주고 갈수록 줄임) (3,10)
    -은닉층 : 입력 10개     출력 30개   (10,30)
    -출력층 : 입력30개      타겟 1개    (30,1)  
  
-손실 함수/ 활성화 함수
    *클래스 형태 ==> nn.MESLoss, nn.ReLU ==> __init__() 메서드에 사용
    *함수 형태 ==> torch.nn.functional => forward()메서드에 사용

In [46]:
class IrisRegModel(nn.Module):

    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()

        self.in_layer=nn.Linear(3, 10)
        self.hidden_layer=nn.Linear(10, 30)
        self.out_layer=nn.Linear(30, 1)
    
    # 순방향 학습 진행 메서드
    def forward(self, input_data):
        # 입력층
        y=self.in_layer(input_data) # f1w1+f2w2+f3w3+b 요런 식이 10개(숫자10개)
        y=F.relu(y)                   # 범위 0이상
        
        # 은닉층 : 10개의 숫자 받아오기
        y=self.hidden_layer(y)
        y=F.relu(y)

        # 출력층 : 30개의 숫자 값
        return self.out_layer(y)
    






In [47]:
# 모델 인스턴스 생성
model = IrisRegModel()
print(model)

# 모델 사용 메모리 정보
summary(model, input_size=(1000,3))

IrisRegModel(
  (in_layer): Linear(in_features=3, out_features=10, bias=True)
  (hidden_layer): Linear(in_features=10, out_features=30, bias=True)
  (out_layer): Linear(in_features=30, out_features=1, bias=True)
)


Layer (type:depth-idx)                   Output Shape              Param #
IrisRegModel                             [1000, 1]                 --
├─Linear: 1-1                            [1000, 10]                40
├─Linear: 1-2                            [1000, 30]                330
├─Linear: 1-3                            [1000, 1]                 31
Total params: 401
Trainable params: 401
Non-trainable params: 0
Total mult-adds (M): 0.40
Input size (MB): 0.01
Forward/backward pass size (MB): 0.33
Params size (MB): 0.00
Estimated Total Size (MB): 0.34

[3] 데이터셋 클래스 설계 및 정의 <hr>
- 데이터셋 : iris.csv
- 피쳐개수 : 3개
- 타겟개수 : 1개
- 클래스이름 : IrisDataSet
- 부모클래스 : utils.data.DataSet
- 속성__필드 : featureDF, targetDF, n_rows, n_featrues  
- 필수 메서드:   
    *__init__(self) : 데이터셋 저장 및 전처리, 개발자가 필요한 속성 설정  
    *__len__(self) : 데이터의 개수 반환  
    *__getItem__(self, index) : 특정 인덱스의 피쳐와 타겟 반환  

In [48]:
class IrisDataset(Dataset):

    def __init__(self, featureDF, targetDF):
        self.featureDF=featureDF
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]
        self.n_features=featureDF.shape[1]
        

    def __len__(self):
        return self.n_rows

    def __getitem__(self, index):
        # 텐서화
        featureTS=torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS=torch.FloatTensor(self.targetDF.iloc[index].values)
        return featureTS, targetTS

In [49]:
# 데이터셋 인스턴스 생성
featureDF = irisdf[irisdf.columns[:-1]] # 2D (150,3)
targetDF = irisdf[irisdf.columns[-1:]] # 1D(150,1)


irisDS=IrisDataset(featureDF,targetDF)


[4] 학습 준비
- 학습 횟수 : EPOCH ( 처음부터 끝까지 공부할 횟수 )
- 배치 크기 : BATCH_SIZE(학습량)
- 위치 지정 : DEVICE (텐서 저장 및 실행 위치 GPU/CPU)
- 학 습 률  : 가중치와 절편 업데이트 시 경사하강법으로 업데이트 간격 설정 0.001~0.1
- 

In [50]:
# 학습 진행 관련 설정 값
EPOCH = 1000
BATCH_SIZE = 10
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
LR = 0.001


- 인스턴스 : 모델, 데이터 셋, 최적화 (, 성능지표) 

In [51]:
# 모델 인스턴스
model = IrisRegModel()

# 데이터셋 인스턴스
X_train, X_test, y_train, y_test = train_test_split(featureDF, targetDF, random_state=1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=1)
print(f'{X_train.shape} {X_test.shape} {X_val.shape}')
print(f'{y_train.shape} {y_test.shape} {y_val.shape}')


trainDS = IrisDataset(X_train, y_train)
testDS = IrisDataset(X_test, y_test)
valDS = IrisDataset(X_val, y_val)

# 데이터 로더 인스턴스
trainDL=DataLoader(trainDS, batch_size=BATCH_SIZE)

# 최적화

(84, 3) (38, 3) (28, 3)
(84, 1) (38, 1) (28, 1)


In [52]:
## 테스트
for feature, target in trainDL:
    print(feature.shape, target.shape)
    break

torch.Size([10, 3]) torch.Size([10, 1])


In [53]:

# 최적화 인스턴스 => w, b model.parameter 전달
optimizer=optim.Adam(model.parameters(), lr=LR)

# 최적화 스케쥴링 인스턴스 생성 ==> LR 조절 및 성능 개선 여부체크
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer,patience=10, verbose=True)



# 손실함수 인스턴스
reqLoss=nn.MSELoss()

c:\Users\KDP-26\anaconda3\envs\TORCH_38\lib\site-packages\torch\optim\lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


[5] 학습 진행

In [54]:
## 학습의 효과 확인 손실값과 성능평가값 저장 필요
LOSS_HISTORY, SCORE_HISTORY=[[],[]], [[],[]]

# 학습 모드로 모델 설정
model.train()

for epoch in range(EPOCH):

    # 학습 모드로 모델 설정
    model.train()

    # 배치크기만큼 데이터 로딩해서 학습 진행
    loss_total, score_total = 0,0

    for featureTS, targetTS in trainDL:

        #학습 진행
        pre_y=model(featureTS)

        #손실 계산
        loss=reqLoss(pre_y, targetTS)
        loss_total += loss.item()


        #성능평가 계산
        score=R2Score()(pre_y, targetTS)
        score_total += score.item()

        #최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    # 에폭당 검증 기능을 키겠다.
    # 모델 검증 모드 설정
    model.eval()
    with torch.no_grad():
        # 검증 데이터셋
        val_featureTS=torch.FloatTensor(valDS.featureDF.values)
        val_targetTS=torch.FloatTensor(valDS.targetDF.values)
        
        #평가
        pre_val=model(val_featureTS)

        #손실
        loss_val=reqLoss(pre_val, val_targetTS)

        # 성능평가
        score_val = R2Score()(pre_val, val_targetTS)




    # 에폭당 손실값과 성능평가 값 저장
    LOSS_HISTORY[0].append(loss_total/BATCH_SIZE)
    SCORE_HISTORY[0].append(score_total/BATCH_SIZE)

    LOSS_HISTORY[1].append(loss_val)
    SCORE_HISTORY[1].append(score_val)

    # 최적화 스케쥴러 인스턴스 업데이트
    scheduler.step(loss_val)
    print(f'scheduler.num_bad_epochs => {scheduler.num_bad_epochs}', end=' ')
    print(f'scheduler.patience => {scheduler.patience}')
    #손실 감소(또는 성능 개선)이 안되는 경우 조기 종료
    if scheduler.num_bad_epochs >= scheduler.patience:
        print(f'{scheduler.patience}EPOCH 성능 개선이 없어서 조기종료함!')
        break

scheduler.num_bad_epochs => 0 scheduler.patience => 10
scheduler.num_bad_epochs => 0 scheduler.patience => 10
scheduler.num_bad_epochs => 0 scheduler.patience => 10
scheduler.num_bad_epochs => 0 scheduler.patience => 10
scheduler.num_bad_epochs => 0 scheduler.patience => 10
scheduler.num_bad_epochs => 0 scheduler.patience => 10
scheduler.num_bad_epochs => 0 scheduler.patience => 10
scheduler.num_bad_epochs => 0 scheduler.patience => 10
scheduler.num_bad_epochs => 0 scheduler.patience => 10
scheduler.num_bad_epochs => 0 scheduler.patience => 10
scheduler.num_bad_epochs => 0 scheduler.patience => 10
scheduler.num_bad_epochs => 0 scheduler.patience => 10
scheduler.num_bad_epochs => 0 scheduler.patience => 10


scheduler.num_bad_epochs => 0 scheduler.patience => 10
scheduler.num_bad_epochs => 0 scheduler.patience => 10
scheduler.num_bad_epochs => 0 scheduler.patience => 10
scheduler.num_bad_epochs => 0 scheduler.patience => 10
scheduler.num_bad_epochs => 0 scheduler.patience => 10
scheduler.num_bad_epochs => 0 scheduler.patience => 10
scheduler.num_bad_epochs => 0 scheduler.patience => 10
scheduler.num_bad_epochs => 0 scheduler.patience => 10
scheduler.num_bad_epochs => 0 scheduler.patience => 10
scheduler.num_bad_epochs => 0 scheduler.patience => 10
scheduler.num_bad_epochs => 0 scheduler.patience => 10
scheduler.num_bad_epochs => 1 scheduler.patience => 10
scheduler.num_bad_epochs => 2 scheduler.patience => 10
scheduler.num_bad_epochs => 3 scheduler.patience => 10
scheduler.num_bad_epochs => 4 scheduler.patience => 10
scheduler.num_bad_epochs => 5 scheduler.patience => 10
scheduler.num_bad_epochs => 6 scheduler.patience => 10
scheduler.num_bad_epochs => 7 scheduler.patience => 10
scheduler.

In [55]:
print(LOSS_HISTORY)
print(SCORE_HISTORY)

[[2.942534160614014, 2.023794746398926, 1.3389312207698822, 0.8624955356121063, 0.5456009894609452, 0.35945998728275297, 0.26695021986961365, 0.2263214871287346, 0.1988663300871849, 0.1751304715871811, 0.1545691967010498, 0.1361067585647106, 0.11953538209199906, 0.10509632527828217, 0.0926325909793377, 0.08194466643035411, 0.07311356291174889, 0.06599867083132267, 0.0604507140815258, 0.05617257729172707, 0.053023144416511056, 0.050837961584329606, 0.049230461940169334, 0.04815588220953941, 0.047480761632323266, 0.046989509277045725, 0.046618917398154734, 0.04633507896214724, 0.04612650442868471, 0.04595104120671749, 0.04581137951463461, 0.045652903988957404, 0.0454736664891243, 0.04530017599463463], [tensor(2.3302), tensor(1.5565), tensor(1.0122), tensor(0.6613), tensor(0.4547), tensor(0.3529), tensor(0.3089), tensor(0.2783), tensor(0.2417), tensor(0.2043), tensor(0.1736), tensor(0.1486), tensor(0.1261), tensor(0.1068), tensor(0.0918), tensor(0.0791), tensor(0.0688), tensor(0.0615), te